In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# -------------------------
# Config: adjust paths here
# -------------------------
DATA_DIR = Path(r"F:\Prj\DS\New folder")   # <- change if your files are elsewhere
OUT_DIR  = Path(r"F:\Prj\DS\New folder\updtaed_new")
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Filenames you mentioned (will try multiple common names)
FILES = {
    "exercise": ["exercise_cleaned.csv", "exercise.csv", "exercise_master.csv"],
    "food_usda": ["food.csv", "food1.csv"],
    "food_indian": ["food_cleaned.csv", "indian_food.csv"],
    "rda": ["rda_cleaned.csv", "rda.csv"],
    "met": ["met.csv", "met_values.csv"],
    "medical_guidelines": ["medical_guidelines.csv"],
    "who_rda": ["who_rda_reference.csv"]
}

def find_file(names):
    for n in names:
        p = DATA_DIR / n
        if p.exists():
            return p
    return None

# -------------------------
# Load helper (safe)
# -------------------------
def safe_read(path, **kwargs):
    try:
        return pd.read_csv(path, **kwargs)
    except Exception as e:
        print(f"Warning: couldn't read {path} -> {e}")
        return pd.DataFrame()

# -------------------------
# 1) Load datasets
# -------------------------
print("Loading files...")
exercise_path = find_file(FILES["exercise"])
food_usda_path = find_file(FILES["food_usda"])
food_indian_path = find_file(FILES["food_indian"])
rda_path = find_file(FILES["rda"])
met_path = find_file(FILES["met"])
medical_path = find_file(FILES["medical_guidelines"])
who_rda_path = find_file(FILES["who_rda"])

print("Paths found:")
print(" exercise:", exercise_path)
print(" food_usda:", food_usda_path)
print(" food_indian:", food_indian_path)
print(" rda:", rda_path)
print(" met:", met_path)
print(" medical:", medical_path)
print(" who_rda:", who_rda_path)

exercise_df = safe_read(exercise_path) if exercise_path else pd.DataFrame()
food_usda_df = safe_read(food_usda_path) if food_usda_path else pd.DataFrame()
food_indian_df = safe_read(food_indian_path) if food_indian_path else pd.DataFrame()
rda_df = safe_read(rda_path) if rda_path else pd.DataFrame()
met_df = safe_read(met_path) if met_path else pd.DataFrame()
medical_df = safe_read(medical_path) if medical_path else pd.DataFrame()
who_rda_df = safe_read(who_rda_path) if who_rda_path else pd.DataFrame()

# -------------------------
# 2) Clean / standardize columns
# -------------------------
def clean_column_names(df):
    df = df.copy()
    df.columns = [str(c).strip().replace("\u200b","") for c in df.columns]
    df.columns = [c.replace("\n"," ").strip() for c in df.columns]
    return df

exercise_df = clean_column_names(exercise_df)
food_usda_df = clean_column_names(food_usda_df)
food_indian_df = clean_column_names(food_indian_df)
rda_df = clean_column_names(rda_df)
met_df = clean_column_names(met_df)
medical_df = clean_column_names(medical_df)
who_rda_df = clean_column_names(who_rda_df)

# --- Exercise: rename expected columns robustly ---
def prepare_exercise(df):
    if df.empty:
        return df
    df = df.copy()
    # find Activity-like col
    possible_activity = [c for c in df.columns if 'activity' in c.lower() or 'title' in c.lower() or 'activity'==c.lower()]
    if possible_activity:
        df = df.rename(columns={possible_activity[0]: "Activity"})
    # find calories-per-kg or equivalent
    possible_cpk = [c for c in df.columns if 'per_kg' in c.lower() or 'calories_per_kg' in c.lower() or 'calories per kg' in c.lower()]
    if possible_cpk:
        df = df.rename(columns={possible_cpk[0]: "Calories_per_kg"})
    # if separate columns for weights, keep them
    for wcol in ["130 lb","Calories_130lb","Calories_130 lb","130lb","Calories_130lb"]:
        if wcol in df.columns:
            df = df.rename(columns={wcol: "Calories_130lb"})
    # coerce numeric
    for col in df.columns:
        if col != "Activity":
            try:
                df[col] = pd.to_numeric(df[col], errors='coerce')
            except:
                pass
    # fill small NaNs
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    df[numeric_cols] = df[numeric_cols].fillna(0)
    return df

exercise_df = prepare_exercise(exercise_df)

# --- METs: normalize
def prepare_met(df):
    if df.empty:
        return df
    df = df.copy()
    # possible activity column
    if 'Description' in df.columns and 'MET' in df.columns:
        df = df.rename(columns={'Description':'Activity','MET':'MET'})
    elif 'ActvitiyCode' in df.columns and 'MET' in df.columns:
        # rename possible misspelling
        df = df.rename(columns={'ActvitiyCode':'Activity','MET':'MET'})
    df['Activity'] = df['Activity'].astype(str).str.strip()
    df['MET'] = pd.to_numeric(df['MET'], errors='coerce').fillna(0)
    return df

met_df = prepare_met(met_df)

# Link met -> exercise by fuzzy match will be done later if possible

# --- Food (USDA-style): extract key nutrient columns when present
def normalize_food_usda(df):
    if df.empty:
        return df
    df = df.copy()
    # Lowercase column names map
    colmap = {c:c for c in df.columns}
    lc = {c.lower():c for c in df.columns}
    # heuristics for calories/protein/carbs/fat/sugar/fibre
    picks = {}
    for target, keywords in {
        "FoodItem": ["description","name","food","category","long_desc"],
        "Calories": ["energy","calories","energy kcal","kcal"],
        "Protein": ["protein"],
        "Carbs": ["carbohydrate","carbs","carbohydrate, by difference"],
        "Fat": ["total lipid","fat"],
        "Sugar": ["sugar","sugars","free sugar"],
        "Fibre": ["fiber","fibre"],
        "Sodium": ["sodium"],
        "Calcium": ["calcium"],
        "Iron": ["iron"],
        "VitaminC": ["vitamin c","vit c"]
    }.items():
        found = None
        for k in keywords:
            if k in lc:
                found = lc[k]; break
        if found:
            picks[target] = found
    # create minimal df
    out = pd.DataFrame()
    if "FoodItem" in picks:
        out["FoodItem"] = df[picks["FoodItem"]].astype(str).str.strip()
    else:
        # try first text-like column
        text_cols = [c for c in df.columns if df[c].dtype == object]
        out["FoodItem"] = df[text_cols[0]].astype(str).str.strip() if text_cols else "unknown"
    for nut in ["Calories","Protein","Carbs","Fat","Sugar","Fibre","Sodium","Calcium","Iron","VitaminC"]:
        if nut in picks:
            out[nut] = pd.to_numeric(df[picks[nut]], errors='coerce')
        else:
            out[nut] = np.nan
    # drop rows with no FoodItem or all NaNs
    out = out.dropna(subset=["FoodItem"]).reset_index(drop=True)
    return out

food_usda_core = normalize_food_usda(food_usda_df)

# --- Food Indian / cleaned: prefer this as primary source for names, calories etc.
def prepare_food_indian(df):
    if df.empty:
        return df
    df = df.copy()
    # Standardize common columns
    col_lc = {c.lower():c for c in df.columns}
    rename_map = {}
    # possible mappings
    if 'dish' in col_lc: rename_map[col_lc['dish']] = 'FoodItem'
    if 'food items' in col_lc: rename_map[col_lc['food items']] = 'FoodItem'
    if 'calories' in col_lc: rename_map[col_lc['calories']] = 'Calories'
    if 'protein' in col_lc: rename_map[col_lc['protein']] = 'Protein'
    if 'carbs' in col_lc: rename_map[col_lc['carbs']] = 'Carbs'
    if 'fat' in col_lc: rename_map[col_lc['fat']] = 'Fat'
    if 'fibre' in col_lc: rename_map[col_lc['fibre']] = 'Fibre'
    if 'sugar' in col_lc: rename_map[col_lc['sugar']] = 'Sugar'
    # apply
    df = df.rename(columns=rename_map)
    # keep minimal columns
    required = ['FoodItem','Calories','Protein','Carbs','Fat','Fibre','Sugar']
    for c in required:
        if c not in df.columns:
            df[c] = np.nan
    df['FoodItem'] = df['FoodItem'].astype(str).str.strip()
    # numeric coercion
    for c in ['Calories','Protein','Carbs','Fat','Fibre','Sugar']:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    return df[required].drop_duplicates(subset=['FoodItem']).reset_index(drop=True)

food_indian_core = prepare_food_indian(food_indian_df)

# --- RDA cleaned dataset: prepare
def prepare_rda(df):
    if df.empty:
        return df
    df = df.copy()
    # unify names
    col_lc = {c.lower():c for c in df.columns}
    rename = {}
    if 'food_items' in col_lc: rename[col_lc['food_items']] = 'FoodItem'
    if 'vegnovveg' in col_lc: rename[col_lc['vegnovveg']] = 'VegNonVeg'
    if 'breakfast' in col_lc: rename[col_lc['breakfast']] = 'Breakfast'
    if 'lunch' in col_lc: rename[col_lc['lunch']] = 'Lunch'
    if 'dinner' in col_lc: rename[col_lc['dinner']] = 'Dinner'
    if 'calories' in col_lc: rename[col_lc['calories']] = 'Calories'
    df = df.rename(columns=rename)
    # standardize VegNonVeg: try to map strings to 0/1
    if 'VegNonVeg' in df.columns:
        df['VegNonVeg'] = df['VegNonVeg'].astype(str).str.strip()
        df['VegNonVeg'] = df['VegNonVeg'].replace({'Veg':0,'NonVeg':1,'Vegetarian':0,'Non Vegetarian':1,'':np.nan})
        # fallback: if col numeric leave as-is
        try:
            df['VegNonVeg'] = pd.to_numeric(df['VegNonVeg'], errors='coerce')
        except:
            pass
    # ensure FoodItem text
    if 'FoodItem' in df.columns:
        df['FoodItem'] = df['FoodItem'].astype(str).str.strip()
    return df

rda_core = prepare_rda(rda_df)

# -------------------------
# 3) Build food_master: merge indian core + usda core (prioritize indian names)
# -------------------------
def build_food_master(indian_core, usda_core):
    # Start with indian core as authoritative for indian dishes
    base = indian_core.copy() if not indian_core.empty else pd.DataFrame()
    # If usda has FoodItem names, merge where names match (simple exact join)
    if not usda_core.empty:
        # dedupe by fooditem in usda
        us = usda_core.drop_duplicates(subset=['FoodItem']).copy()
        # left-join on FoodItem
        if not base.empty:
            merged = pd.merge(base, us, on='FoodItem', how='left', suffixes=('_ind','_us'))
        else:
            merged = us.copy()
        # fill missing nutrition columns using usda data if indian lacks them
        for col in ['Calories','Protein','Carbs','Fat','Fibre','Sugar','Sodium','Calcium','Iron','VitaminC']:
            col_ind = col + '_ind' if col + '_ind' in merged.columns else col
            col_us = col + '_us' if col + '_us' in merged.columns else col
            if col_ind in merged.columns and col_us in merged.columns:
                merged[col] = merged[col_ind].fillna(merged[col_us])
            elif col in merged.columns:
                merged[col] = merged[col]
            else:
                merged[col] = np.nan
        # ensure FoodItem exists
        if 'FoodItem' not in merged.columns and 'FoodItem' in us.columns:
            merged['FoodItem'] = us['FoodItem']
        # keep canonical cols
        keep = ['FoodItem','Calories','Protein','Carbs','Fat','Fibre','Sugar','Sodium','Calcium','Iron','VitaminC']
        for k in keep:
            if k not in merged.columns:
                merged[k] = np.nan
        master = merged[keep].drop_duplicates(subset=['FoodItem']).reset_index(drop=True)
    else:
        master = base.copy()
    # final cleaning
    for c in ['Calories','Protein','Carbs','Fat','Fibre','Sugar','Sodium','Calcium','Iron','VitaminC']:
        if c in master.columns:
            master[c] = pd.to_numeric(master[c], errors='coerce')
    master = master.drop_duplicates(subset=['FoodItem']).reset_index(drop=True)
    return master

food_master = build_food_master(food_indian_core, food_usda_core)

# If food_master is empty, fall back to any available food df
if food_master.empty and not food_indian_df.empty:
    food_master = prepare_food_indian(food_indian_df)

# -------------------------
# 4) Build exercise_master: map MET or calories_per_kg
# -------------------------
def build_exercise_master(ex_df, met_df):
    if ex_df.empty:
        return ex_df
    ex = ex_df.copy()
    # Ensure Activity exists
    if 'Activity' not in ex.columns:
        possible = [c for c in ex.columns if 'title' in c.lower() or 'name' in c.lower()]
        if possible:
            ex = ex.rename(columns={possible[0]:'Activity'})
    ex['Activity'] = ex['Activity'].astype(str).str.strip()
    # If Calories_per_kg exists use it; else attempt to compute from MET
    if 'Calories_per_kg' not in ex.columns or ex['Calories_per_kg'].isna().all():
        # try to join on Activity with METs and compute calories_per_kg = MET * 1.05 (approx)
        if not met_df.empty:
            # simple exact join first
            merged = pd.merge(ex, met_df[['Activity','MET']].drop_duplicates(), on='Activity', how='left')
            if 'MET' in merged.columns:
                merged['Calories_per_kg'] = merged['MET'] * 1.05  # conversion rule used earlier
            ex = merged
    # fill numeric
    ex['Calories_per_kg'] = pd.to_numeric(ex.get('Calories_per_kg', pd.Series(np.nan)), errors='coerce').fillna(0)
    # Infer category column if missing
    if 'BodyPart' in ex.columns:
        ex['Category'] = ex['BodyPart']
    else:
        # keyword-based
        cardio_kw = ['run','cycle','swim','cardio','aerobic','stair','ski']
        strength_kw = ['barbell','kettlebell','weight','press','squat','deadlift','bench','lift','curl','pull','push']
        def detect_cat(a):
            a_low = str(a).lower()
            if any(k in a_low for k in cardio_kw): return 'Cardio'
            if any(k in a_low for k in strength_kw): return 'Strength'
            return 'Mixed'
        ex['Category'] = ex['Activity'].apply(detect_cat)
    # drop duplicates and return
    return ex[['Activity','Calories_per_kg','Category']].drop_duplicates(subset=['Activity']).reset_index(drop=True)

exercise_master = build_exercise_master(exercise_df, met_df)

# -------------------------
# 5) Integrate medical_guidelines & who_rda (just load)
# -------------------------
medical_master = medical_df.copy() if not medical_df.empty else pd.DataFrame()
who_rda_master = who_rda_df.copy() if not who_rda_df.empty else pd.DataFrame()

# -------------------------
# 6) Save outputs and print checks
# -------------------------
food_master.to_csv(OUT_DIR / "food_master.csv", index=False)
exercise_master.to_csv(OUT_DIR / "exercise_master.csv", index=False)
medical_master.to_csv(OUT_DIR / "medical_guidelines_processed.csv", index=False)
who_rda_master.to_csv(OUT_DIR / "who_rda_processed.csv", index=False)

print("\nSaved processed files to:", OUT_DIR)
print("Food master shape:", food_master.shape)
print("Exercise master shape:", exercise_master.shape)
print("Medical rules rows:", 0 if medical_master.empty else medical_master.shape[0])
print("WHO RDA rows:", 0 if who_rda_master.empty else who_rda_master.shape[0])

# Quick head previews
print("\n--- Food master sample ---")
print(food_master.head(5).to_string(index=False))
print("\n--- Exercise master sample ---")
print(exercise_master.head(8).to_string(index=False))

# Sanity checks
if food_master['FoodItem'].duplicated().any():
    print("Warning: duplicate food items present.")
if exercise_master['Activity'].duplicated().any():
    print("Warning: duplicate exercise activities present.")


Loading files...
Paths found:
 exercise: F:\Prj\DS\New folder\exercise_cleaned.csv
 food_usda: F:\Prj\DS\New folder\food.csv
 food_indian: F:\Prj\DS\New folder\food_cleaned.csv
 rda: F:\Prj\DS\New folder\rda_cleaned.csv
 met: F:\Prj\DS\New folder\met.csv
 medical: F:\Prj\DS\New folder\medical_guidelines.csv
 who_rda: F:\Prj\DS\New folder\who_rda_reference.csv

Saved processed files to: F:\Prj\DS\New folder\updtaed_new
Food master shape: (1014, 11)
Exercise master shape: (248, 3)
Medical rules rows: 12
WHO RDA rows: 8

--- Food master sample ---
                   FoodItem  Calories  Protein  Carbs  Fat  Fibre  Sugar  Sodium  Calcium  Iron  VitaminC
       Hot tea (Garam Chai)     16.14     0.39   2.58 0.53   0.00   2.58     NaN      NaN   NaN       NaN
             Instant coffee     23.16     0.64   3.65 0.75   0.00   3.62     NaN      NaN   NaN       NaN
             Espreso coffee     51.54     1.75   6.62 2.14   0.00   6.53     NaN      NaN   NaN       NaN
                   Iced t

In [4]:
# ---------------------------------------------------
# Step 2: Feature Engineering - Prepare Model Inputs
# ---------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)


# -------------------------
# Path Configuration
# -------------------------
DATA_DIR = Path(r"F:\Prj\DS\New folder\updtaed_new")
OUT_DIR = DATA_DIR / "model_ready"
OUT_DIR.mkdir(parents=True, exist_ok=True)

# -------------------------
# Load Processed Files
# -------------------------
food_master = pd.read_csv(DATA_DIR / "food_master.csv")
exercise_master = pd.read_csv(DATA_DIR / "exercise_master.csv")
medical_df = pd.read_csv(DATA_DIR / "medical_guidelines_processed.csv")
who_rda_df = pd.read_csv(DATA_DIR / "who_rda_processed.csv")

print("✅ Datasets Loaded Successfully")
print(f"Food shape: {food_master.shape}, Exercise shape: {exercise_master.shape}")

# -------------------------
# Normalize Food Nutrients
# -------------------------
nutrient_cols = ['Calories', 'Protein', 'Carbs', 'Fat', 'Fibre', 'Sugar']
food_master[nutrient_cols] = food_master[nutrient_cols].fillna(0)

scaler = MinMaxScaler()
food_master_scaled = food_master.copy()
food_master_scaled[nutrient_cols] = scaler.fit_transform(food_master[nutrient_cols])

# -------------------------
# Label Encode Category / Type
# -------------------------
exercise_master['Category'] = exercise_master['Category'].replace({
    'Cardio': 0, 'Strength': 1, 'Mixed': 2
})

# -------------------------
# Feature Summary Statistics
# -------------------------
food_summary = food_master_scaled.describe().T
exercise_summary = exercise_master.describe().T

print("\n--- Food Nutrient Summary ---")
print(food_summary[['min','max','mean']])
print("\n--- Exercise Feature Summary ---")
print(exercise_summary[['min','max','mean']])

# -------------------------
# Save Model-Ready Datasets
# -------------------------
food_master_scaled.to_csv(OUT_DIR / "food_features.csv", index=False)
exercise_master.to_csv(OUT_DIR / "exercise_features.csv", index=False)
medical_df.to_csv(OUT_DIR / "medical_guidelines_features.csv", index=False)
who_rda_df.to_csv(OUT_DIR / "who_rda_features.csv", index=False)

print("\n✅ Feature Engineering Completed Successfully")
print(f"Saved model-ready files to: {OUT_DIR}")


✅ Datasets Loaded Successfully
Food shape: (1014, 11), Exercise shape: (248, 3)

--- Food Nutrient Summary ---
          min  max      mean
Calories  0.0  1.0  0.272752
Protein   0.0  1.0  0.220557
Carbs     0.0  1.0  0.212111
Fat       0.0  1.0  0.179852
Fibre     0.0  1.0  0.054956
Sugar     0.0  1.0  0.103130
Sodium    NaN  NaN       NaN
Calcium   NaN  NaN       NaN
Iron      NaN  NaN       NaN
VitaminC  NaN  NaN       NaN

--- Exercise Feature Summary ---
                      min       max      mean
Calories_per_kg  0.310067  3.706591  1.359930
Category         0.000000  2.000000  1.516129

✅ Feature Engineering Completed Successfully
Saved model-ready files to: F:\Prj\DS\New folder\updtaed_new\model_ready


In [6]:
# ---------------------------------------------------
# Step 3: Personalized Diet & Exercise Recommendation Engine
# ---------------------------------------------------
import pandas as pd
import numpy as np
from pathlib import Path

# -------------------------
# Paths
# -------------------------
DATA_DIR = Path(r"F:\Prj\DS\New folder\updtaed_new\model_ready")

food_df = pd.read_csv(DATA_DIR / "food_features.csv")
exercise_df = pd.read_csv(DATA_DIR / "exercise_features.csv")
medical_df = pd.read_csv(DATA_DIR / "medical_guidelines_features.csv")
who_rda_df = pd.read_csv(DATA_DIR / "who_rda_features.csv")

print("✅ All datasets loaded successfully!")

# -------------------------
# Utility Functions
# -------------------------
def calculate_tdee(age, gender, height, weight, activity_level):
    """Calculate Total Daily Energy Expenditure (TDEE)."""
    if gender.lower() == 'male':
        bmr = 10 * weight + 6.25 * height - 5 * age + 5
    else:
        bmr = 10 * weight + 6.25 * height - 5 * age - 161

    if activity_level == 0:
        factor = 1.2
    elif activity_level <= 3:
        factor = 1.375
    elif activity_level <= 5:
        factor = 1.55
    else:
        factor = 1.725

    return bmr * factor

def recommend_diet(goal, diet_pref, calorie_target, condition=None):
    """Generate top diet recommendations based on user inputs."""
    df = food_df.copy()

    # 1️⃣ Goal-based calorie adjustment
    if goal.lower() == "lose weight":
        target = calorie_target * 0.85
    elif goal.lower() == "gain weight":
        target = calorie_target * 1.15
    else:
        target = calorie_target

    lower, upper = target * 0.85 / 3, target * 1.15 / 3
    df = df[(df['Calories'] >= lower) & (df['Calories'] <= upper)]

    # 2️⃣ Diet preference
    if "VegNonVeg" in df.columns:
        if diet_pref.lower() == "veg":
            df = df[df["VegNonVeg"] == 0]
        elif diet_pref.lower() == "nonveg":
            df = df[df["VegNonVeg"] == 1]
    else:
        # fallback to keyword match if column not present
        if diet_pref.lower() == 'veg':
            df = df[~df['FoodItem'].str.contains('chicken|meat|fish|egg', case=False, na=False)]
        elif diet_pref.lower() == 'nonveg':
            df = df[df['FoodItem'].str.contains('chicken|meat|fish|egg', case=False, na=False)]

    # 3️⃣ Apply medical conditions if provided
    if condition and condition in medical_df['Condition'].values:
        avoid = medical_df.loc[medical_df['Condition']==condition, 'Avoid'].values[0]
        rec = medical_df.loc[medical_df['Condition']==condition, 'Recommend'].values[0]
        df = df[~df['FoodItem'].str.contains('|'.join(avoid.split(',')), case=False, na=False)]
        df = df[df['FoodItem'].str.contains('|'.join(rec.split(',')), case=False, na=False)]

    # 4️⃣ Rank by protein and nutrient balance
    df["NutrientScore"] = (
        0.4*df["Protein"] + 
        0.2*df["Fibre"] - 
        0.2*df["Sugar"] - 
        0.2*df["Fat"]
    )
    df = df.sort_values(by="NutrientScore", ascending=False).head(5)

    return df[['FoodItem','Calories','Protein','Carbs','Fat','Fibre','Sugar']]


def recommend_exercises(goal):
    """Return top exercises based on user goal."""
    ex = exercise_df.copy()
    if goal.lower() == "lose weight":
        ex = ex[ex['Category']==0]  # Cardio
    elif goal.lower() == "gain weight":
        ex = ex[ex['Category']==1]  # Strength
    else:
        ex = ex[ex['Category'].isin([0,1])]  # Mixed

    ex = ex.sort_values(by='Calories_per_kg', ascending=False).head(5)
    
    # Add YouTube demo links (manual mapping or CSV later)
    yt_links = {
        'Running': 'https://www.youtube.com/watch?v=brmG9NvG8jk',
        'Cycling': 'https://www.youtube.com/watch?v=Hu4Yvq-g7_Y',
        'Push-Ups': 'https://www.youtube.com/watch?v=_l3ySVKYVJ8',
        'Plank': 'https://www.youtube.com/watch?v=pSHjTRCQxIw',
        'Squats': 'https://www.youtube.com/watch?v=aclHkVaku9U'
    }

    ex['YouTubeDemo'] = ex['Activity'].apply(
        lambda x: yt_links.get(x.split()[0].capitalize(), f"https://www.youtube.com/results?search_query={x.replace(' ','+')}")
    )

    return ex[['Activity','Calories_per_kg','Category','YouTubeDemo']]

# -------------------------
# Example Run
# -------------------------
user = {
    "age": 25,
    "gender": "Male",
    "height": 175,
    "weight": 70,
    "activity_level": 4,
    "goal": "Lose Weight",
    "diet_pref": "Veg",
    "condition": "Diabetes"
}

user_tdee = calculate_tdee(user['age'], user['gender'], user['height'], user['weight'], user['activity_level'])
print(f"\n🔥 Estimated TDEE: {user_tdee:.2f} kcal/day")

diet_recs = recommend_diet(user['goal'], user['diet_pref'], user_tdee, user['condition'])
exercise_recs = recommend_exercises(user['goal'])

print("\n🍽️ Top Diet Recommendations:")
print(diet_recs)

print("\n🏋️ Top Exercise Recommendations:")
print(exercise_recs)


✅ All datasets loaded successfully!

🔥 Estimated TDEE: 2594.31 kcal/day

🍽️ Top Diet Recommendations:
Empty DataFrame
Columns: [FoodItem, Calories, Protein, Carbs, Fat, Fibre, Sugar]
Index: []

🏋️ Top Exercise Recommendations:
                             Activity  Calories_per_kg  Category  \
47   Running, 10.9 mph (5.5 min mile)         3.706591         0   
217      Cross country skiing, uphill         3.397878         0   
188                 Skin diving, fast         3.294974         0   
46       Running, 10 mph (6 min mile)         3.294974         0   
45      Running, 9 mph (6.5 min mile)         3.089165         0   

                                           YouTubeDemo  
47   https://www.youtube.com/results?search_query=R...  
217  https://www.youtube.com/results?search_query=C...  
188  https://www.youtube.com/results?search_query=S...  
46   https://www.youtube.com/results?search_query=R...  
45   https://www.youtube.com/results?search_query=R...  
